In [1]:
import datetime
import os
import pandas as pd
import numpy 
from haversine import haversine
import json
import numpy as np
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [2]:
sotf_excel=pd.ExcelFile("./BL SOTF November Zips - Request.xlsx")
sotf_excel.sheet_names

['BL_11.9 Zips_77 Stores', 'BL_11.23 Zips_4 Stores']

In [3]:
df_sotf_1109=sotf_excel.parse('BL_11.9 Zips_77 Stores',dtype=str)
df_sotf_1123=sotf_excel.parse('BL_11.23 Zips_4 Stores',dtype=str)

df_sotf_1109.shape,df_sotf_1123.shape

((77, 9), (4, 9))

In [4]:
len(set(df_sotf_1109['location_id'].tolist()+df_sotf_1123['location_id'].tolist()))

81

In [5]:
TA_excel=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_remove_8_and_22_stores_JL_2019-09-11.xlsx")
store_ta_zips=TA_excel.parse("view_by_store",dtype=str)



In [6]:
store_list_1109=df_sotf_1109['location_id'].unique().tolist()
store_list_1123=df_sotf_1123['location_id'].unique().tolist()

all_sotf=store_list_1109+store_list_1123
df_existing=store_ta_zips[store_ta_zips['location_id'].isin(all_sotf)]

In [7]:
store_list_new=[x for x in all_sotf if x not in df_existing['location_id'].tolist()]
print(store_list_new)
print(len(store_list_new))

['5399', '5400', '4711', '5411', '4709', '5410', '4707', '4701', '5409']
9


In [8]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20191001-134050-499.txt",dtype=str,sep="|")
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
store_list['zip_cd']=store_list['zip_cd'].apply(lambda x: x.strip().split("-")[0].zfill(5))
store_list=store_list.rename(columns={"zip_cd":"store_zip"})
store_list[store_list['location_id'].isin(store_list_new)]
# Empty df=all not opened stores

,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,store_zip,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet


In [9]:
# Google the location for new stores
dict_new_store_address={}

# 5399 	851 SOUTH 30TH STREET	INDIAN MOUND MALL	HEATH	OH	43056
dict_new_store_address.update({"5399":{"GoogleAddress":"771 S 30th St Ste 9001, Heath, OH 43056","GoogleCoor":[40.0312042,-82.4473616]}})

# 5400 	2235 EAST STATE STREET	HERMITAGE TOWN PLAZA	HERMITAGE	PA	16148-2727
dict_new_store_address.update({"5400":{"GoogleAddress":"2253 E State St, Hermitage, PA 16148","GoogleCoor":[41.234349,-80.4658798]}})

# 4711 	4655 CANAL AVENUE SOUTHWEST	GRANDVILLE MARKETPLACE	GRANDVILLE	MI	49418-8731
dict_new_store_address.update({"4711":{"GoogleAddress":"4485-4655 Canal Ave SW, Grandville, MI 49418","GoogleCoor":[42.8815927,-85.7745291]}})

# 5411 	2318 WEST MERCURY BOULEVARD	TODD'S LANE SHOPS	HAMPTON	VA	23666
dict_new_store_address.update({"5411":{"GoogleAddress":"2318 W Mercury Blvd, Hampton, VA 23666","GoogleCoor":[37.0394621,-76.4073108]}})

# 4709 	5252 BARDSTOWN ROAD	BARDSTOWN PLACE	LOUISVILLE	KY	40291-1766
dict_new_store_address.update({"4709":{"GoogleAddress":"5252 Bardstown Rd, Louisville, KY 40291","GoogleCoor":[38.1680876,-85.6073362]}})

# 5410 	9690 COLERAIN AVENUE	PROSPECT SQUARE	CINCINNATI	OH	45251
dict_new_store_address.update({"5410":{"GoogleAddress":"9690 Colerain Ave, Cincinnati, OH 45251","GoogleCoor":[39.2482448,-84.5948913]}})

# 4707 	14215 SOUTHEAST PETROVITSKY ROAD	FAIRWOOD SQUARE	RENTON	WA	98058-8983
dict_new_store_address.update({"4707":{"GoogleAddress":"14215 SE Petrovitsky Rd, Renton, WA 98058","GoogleCoor":[47.4444178,-122.1519844]}})

# 4701 	5055 W SAHARA AVE	SAHARA DECATUR	LAS VEGAS	NV	89146-3405
dict_new_store_address.update({"4701":{"GoogleAddress":"5055 W Sahara Ave, Las Vegas, NV 89146","GoogleCoor":[36.1436122,-115.2103361]}})

# 5409 	4585 EASTGATE BOULEVARD	TOYS R US CENTER	CINCINNATI	OH	45245-1201
dict_new_store_address.update({"5409":{"GoogleAddress":"4585 Eastgate Blvd, Cincinnati, OH 45245","GoogleCoor":[39.102485,-84.2740534]}})



In [10]:
df_output_existing=store_list[['location_id','state_nm','store_zip','latitude_meas','longitude_meas']]
df_output_existing=df_output_existing[df_output_existing['location_id'].isin(df_existing['location_id'].unique().tolist())]
df_zip_cd=pd.DataFrame()
df_existing=df_existing.reset_index()
del df_existing['index']
for ind,row in df_existing.iterrows():
    store_num=row['location_id']
    zip_list_P=eval(row['trans_P_zips_70_within_TA'])
    zip_list_S=eval(row['trans_S_zips_70_within_TA'])
    zip_list_10=eval(row['zips_in_10'])
    
    df_P=pd.DataFrame({"zip":zip_list_P},index=["P"]*len(zip_list_P))
    df_S=pd.DataFrame({"zip":zip_list_S},index=["S"]*len(zip_list_S))
    df_10=pd.DataFrame({"zip":zip_list_10},index=["zip_in_10"]*len(zip_list_10))
    
    df=pd.concat([df_P,df_S,df_10]).reset_index().rename(columns={"index":"trans_flag"})
    df['location_id']=store_num
    df_zip_cd=df_zip_cd.append(df)
df_output_existing=pd.merge(df_output_existing,df_zip_cd,on="location_id",how="left")

In [11]:
df_output_existing[(df_output_existing['latitude_meas']==0) | (df_output_existing['longitude_meas']==0)]['location_id'].unique().tolist()

[]

In [12]:
df_output_existing['Distance (mile)']=np.nan
df_output_existing=df_output_existing.reset_index()
del df_output_existing['index']

for ind, row in df_output_existing.iterrows():
    dist=haversine((row['latitude_meas'],row['longitude_meas']),zip_centers[row['zip']],miles=True)
    df_output_existing.loc[ind,"Distance (mile)"]=dist
    

In [13]:
df_output_new=pd.DataFrame()
for location_id in store_list_new:
    state_nm=dict_new_store_address[location_id]['GoogleAddress'].split(", ")[-1][:2]
    store_zip=dict_new_store_address[location_id]['GoogleAddress'].split(", ")[-1].split(" ")[1]
    latitude_meas=dict_new_store_address[location_id]['GoogleCoor'][0]
    longitude_meas=dict_new_store_address[location_id]['GoogleCoor'][1]
    df=pd.DataFrame()
    for zip_cd,v in zip_centers.items():
        dist=haversine((latitude_meas,longitude_meas),v,miles=True)
        if dist<=10:
            df=df.append(pd.DataFrame({"zip":zip_cd,"Distance (mile)":dist},index=["zip_in_10"]))
    df=df.reset_index().rename(columns={"index":"trans_flag"})
    df['location_id']=location_id
    df['state_nm']=state_nm
    df['store_zip']=store_zip
    df['latitude_meas']=latitude_meas
    df['longitude_meas']=longitude_meas
    df_output_new=df_output_new.append(df)
    
            

In [15]:
df_output_all=df_output_existing.append(df_output_new)
df_output_all['G.O.']=np.nan
col_list=df_sotf_1109.columns.tolist()

col_list_new=[]
for col in col_list:
    if col=="revenue_flag":
        new_col="trans_flag"
    else:
        new_col=col
    col_list_new=col_list_new+[new_col]
df_output_all=df_output_all[col_list_new]

In [16]:
df_output_all['G.O.']=np.where(df_output_all['location_id'].isin(store_list_1109),"1109",
                              np.where(df_output_all['location_id'].isin(store_list_1123),"1123",np.nan)
                              )
df_output_1109=df_output_all[df_output_all['G.O.']=="1109"]
df_output_1123=df_output_all[df_output_all['G.O.']=="1123"]


In [17]:
df_unique_zips_1109=df_output_1109[['trans_flag','zip']].sort_values(["zip","trans_flag"]).drop_duplicates("zip")
df_unique_zips_1123=df_output_1123[['trans_flag','zip']].sort_values(["zip","trans_flag"]).drop_duplicates("zip")

In [18]:
writer=pd.ExcelWriter("./BL SOTF November Zips_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_zips_1109.to_excel(writer,'unique_zip_1109',index=False)
df_unique_zips_1123.to_excel(writer,'unique_zip_1123',index=False)

df_output_1109.to_excel(writer,'BL_11.9 Zips_77 Stores',index=False)
df_output_1123.to_excel(writer,'BL_11.23 Zips_4 Stores',index=False)
writer.save()